In [1]:
# ID: dVDeyM_LCOl2tFQpIWa2lg
# Password (secret): hKWN3XoJXlXCmA_QILUG6K6KSKx44A
# Install official Reddit API wrapper
# !pip install praw 
# We now want to import the usual suspects 
import pandas as pd 
import numpy as np
import re 
import praw

In [2]:
user_id = 'Comment Scraping (Sentiment Analysis) by /u/angry_fellow'
reddit = praw.Reddit(
    client_id = 'dVDeyM_LCOl2tFQpIWa2lg',
    client_secret = 'hKWN3XoJXlXCmA_QILUG6K6KSKx44A',
    user_agent = user_id)

In [3]:
# specify subreddit:
subreddit = reddit.subreddit('2007scape')
# Can look through categories ~ but are we really interested in that?
# Set the url: https://www.reddit.com/r/2007scape/comments/11ltva6/ruinous_powers_v11_updated_greenlight_poll_blog/
post = reddit.submission(url = 'https://www.reddit.com/r/2007scape/comments/11ltva6/ruinous_powers_v11_updated_greenlight_poll_blog/')


In [4]:
# We now wish to loop and to gather all of the comments made...
conversedict = {}
post.comments.replace_more(limit=100)
for comment in post.comments.list():
    
    if comment.id not in conversedict:
        conversedict[comment.id] = [[comment.body, comment.ups],{}]
        if comment.parent() != post.id:
                    parent = str(comment.parent())
                    conversedict[parent][1][comment.id] = [comment.ups, comment.body]
print(conversedict)    

{'jbe3ase': [['This seems to address a lot of the feedback so far.\n\nThere are some really nice changes in here that add what sound like rewarding skill ceiling gameplay elements, and I think the new elemental prayers are a cool thematic inclusion. Would the blood effect work with all combat styles..?\n\nI think at this point actually testing the 90% dmg reduction prayers in a beta is kinda essential because all this conjecture about how "awful" it\'d be is pretty baseless (outside of ToA but we all know that invocation is mostly shit because it\'s underrewarding in terms of points for the inconvenience).', 250], {'jbe3kcn': [97, "Cruor's Vow would work with all combat styles yeah, I'll update the blog to reflect as much!"], 'jbe8g6l': [6, 'During the initial proposal I was like: hmm, fine, whatever. But this iteration got me quite hyped really'], 'jbfo38h': [1, 'Given the power of the book, I don’t think we can conclude it’ll feel awful in ToA. 500s are still very possible with the p

In [5]:
print(conversedict['jbe3ase'])    

[['This seems to address a lot of the feedback so far.\n\nThere are some really nice changes in here that add what sound like rewarding skill ceiling gameplay elements, and I think the new elemental prayers are a cool thematic inclusion. Would the blood effect work with all combat styles..?\n\nI think at this point actually testing the 90% dmg reduction prayers in a beta is kinda essential because all this conjecture about how "awful" it\'d be is pretty baseless (outside of ToA but we all know that invocation is mostly shit because it\'s underrewarding in terms of points for the inconvenience).', 250], {'jbe3kcn': [97, "Cruor's Vow would work with all combat styles yeah, I'll update the blog to reflect as much!"], 'jbe8g6l': [6, 'During the initial proposal I was like: hmm, fine, whatever. But this iteration got me quite hyped really'], 'jbfo38h': [1, 'Given the power of the book, I don’t think we can conclude it’ll feel awful in ToA. 500s are still very possible with the prayer invo o

In [6]:
# We now convert this into a dataframe ~ userid is not necessarily a key keep that in mind 
uid_list = []
comment_list = []
upvotes_list = []
for item in conversedict:
    # for parent comment
    uid_list.append(item)
    comment_list.append(conversedict[item][0][0])
    upvotes_list.append(conversedict[item][0][1])
    # for replies
    for item1 in conversedict[item][1]:
        uid_list.append(item1)
        comment_list.append(conversedict[item][1][item1][1])
        upvotes_list.append(conversedict[item][1][item1][0])
# convert it to a df:
reddit_comments_df = pd.DataFrame(list(zip(uid_list, comment_list, upvotes_list)), columns = ['User ID', 'Comments', 'Upvotes'])
reddit_comments_df.head()

,User ID,Comments,Upvotes
0,jbe3ase,This seems to address a lot of the feedback so...,250
1,jbe3kcn,Cruor's Vow would work with all combat styles ...,97
2,jbe8g6l,"During the initial proposal I was like: hmm, f...",6
3,jbfo38h,"Given the power of the book, I don’t think we ...",1
4,jbe3exk,"For Glacies' Vow, is the bonus magic damage de...",72


In [7]:
reddit_comments_df.to_csv(r'C:\Users\huang\OneDrive\Documents\ECO481\text_df.csv')